In [1]:
!pip install --upgrade --quiet pip 
!pip install -I --upgrade --quiet setuptools
!pip install --upgrade --quiet pymongo
!pip install --quiet lightgbm

In [1]:
% matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
import modelling.fetch as fetch
from modelling.models import *
from modelling.utils import get_articles
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from pymongo import MongoClient
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer
import lightgbm

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
def number(n):
    try:
        float(n)
        return True
    except:
        return False

df[df.cluster.map(number)].sort_values('added')

In [ ]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(5, shuffle = True)

def evaluate(model, tfidf, y, method = 'predict_proba', quiet = False):
    # ans = [1 if l == True else False for l in y]
    ans = y
    probs = cross_val_predict(model, tfidf, ans, cv=cv, method=method)
    predictions = cross_val_predict(model, tfidf, ans, cv=cv, method='predict')

    if method == 'predict_proba':
        probs = [p[1] for p in probs] 

    if not quiet:
        print (average_precision_score(ans, probs), roc_auc_score(ans, probs, average = None))
        print (precision_score(ans, predictions), recall_score(ans, predictions), fbeta_score(ans, predictions, beta = 1.5))
    # precision, recall, _ = precision_recall_curve(ans, guess)
    fpr, tpr, _ = roc_curve(ans, probs)
    plt.step(fpr, tpr)
    # plt.step(precision, recall)
    return probs, ans, predictions

In [ ]:
scorer = make_scorer(fbeta_score, beta = 1.5, pos_label = 'accepted')
parameters = [{ }]
grid = GridSearchCV(lightgbm.LGBMClassifier(n_estimators = 100, reg_lambda = 0.6, reg_alpha = 0.2), parameters, scoring = scorer)
# p,a,pr = evaluate(model, tfidf, df)
grid.fit(word_count, df.label)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

In [ ]:
tfidf = vectorizer(X)

model = lightgbm.LGBMClassifier(n_estimators = 10, reg_lambda = 0.6, reg_alpha = 0.2)
_ = evaluate(model, word_count, df)

In [ ]:
scorer = make_scorer(fbeta_score, beta = 1.5, pos_label = 'accepted')
parameters = [{ }]
model = svm.LinearSVC(dual = False, class_weight = 'balanced', C = 4., tol = 10e-6, max_iter = 4000)
grid = GridSearchCV(model, parameters, scoring = scorer)
# p,a,pr = evaluate(model, tfidf, df)
grid.fit(word_count, df.label)
pd.DataFrame(grid.cv_results_).sort_values('rank_test_score')

In [ ]:
from sklearn import svm

svc = svm.LinearSVC(dual = False,  tol = 10e-6, max_iter = 10000)
g,a,p = evaluate(svc, tfidf, df, 'decision_function')

In [ ]:
nb = MultinomialNB(class_prior = [.4,.6])
g, ans, preds = evaluate(nb, tfidf, df)

In [3]:
collection = MongoClient('209.177.92.45:80')['newsfilter'].news

In [4]:
X, y, ids = get_prediction_data(collection, label=True)

In [14]:
df = get_prediction_data(collection, label=True, whole_df = True)

In [17]:
s = df.sort_values('added').reset_index(drop=True)

In [21]:
X,y = s.text, s.label

In [28]:
start = 0
cutoff = 4500
end = ids.shape[0]

X_train, y_train = X[start:cutoff], y[start:cutoff]
X_test, y_test = X[cutoff:end], y[cutoff:end]

In [29]:
model = base_model()
base_preds = train_and_predict(model, X_train, y_train, X_test)

In [30]:
p = base_preds > .5
print (precision_score(y_test, p), recall_score(y_test, p), fbeta_score(y_test, p, beta = 1.5))
p.sum() / y_test.shape[0]

0.514705882353 0.478142076503 0.488826815642


0.1693227091633466